In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 3})
sns.set_style('whitegrid',
              {'grid.linestyle': ':', 'grid.color': 'red', 'axes.edgecolor': '0.5',
               'axes.linewidth': 1.2, 'legend.frameon': True})

import os
import pickle

In [2]:
from scipy.constants import e, m_p, c

from scipy.constants import physical_constants

In [3]:
from cpymad.madx import Madx

import pysixtracklib as pyst

In [4]:
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3
nmass = 0.931494061 # MAD-X value

### $\implies$ hard coded apertures in PySixTrackLib!!!

### $\implies$ missing dx, dy and dphi errors still!

In [5]:
tune_range_qx = np.arange(18.55, 18.95 + 0.01, 0.01)
tune_range_qy = tune_range_qx

# prepare MAD-X error sequences

In [6]:
e_seed = 1

for qx in tune_range_qx:
    for qy in tune_range_qy:
        qqx, qqy = int(np.round((qx%1) * 100)), int(np.round((qy%1) * 100))
        
        filename_error_table = "./errors_{qqx}_{qqy}_{eseed:d}".format(
            qqx=qqx, qqy=qqy, eseed=e_seed)
        
        if os.path.exists(filename_error_table + '_full-lattice.seq'):
            continue

        print ('\n\n\n=== Preparing for Qx = {:.2f} and Qy = {:.2f} ===\n\n\n'.format(qx, qy))

        madx = Madx()
        madx.options.echo = False
        madx.options.warn = False
        
        madx.call('./SIS100_RF_220618_9slices.thin.seq')
        
        madx.command.beam(particle='ion', mass=238*nmass, charge=28, energy=238*0.2+238*nmass)
        
        madx.call('OpticsYEH_BeamParameters.str')
        madx.call('Coll+Errors+BeamDistr.madx')
        
        madx.use(sequence='sis100ring')
        
        madx.input('''
            eoption, add=true, seed = {eseed:d};
            exec,EA_EFCOMP_MH();
            
            slice = 1;
            while (slice<=9)
            {{
            exec,EA_rEFCOMP_QD($slice,1);
            slice=slice+1;
            }}

            SELECT, FLAG = ERROR, PATTERN=QD11.., class=MULTIPOLE;
            SELECT, FLAG = ERROR, PATTERN=QD12.., class=MULTIPOLE;
            SELECT, FLAG = ERROR, PATTERN=mh1, class=MULTIPOLE;
            SELECT, FLAG = ERROR, PATTERN=mh2, class=MULTIPOLE;
            ESAVE, FILE = "errors_{qqx}_{qqy}_{eseed:d}";
            
            cavity_voltage = 58.2/1000/number_cavities;
        '''.format(eseed=e_seed, qx=qx, qy=qy, qqx=qqx, qqy=qqy))
        
        madx.use(sequence='sis100ring')
#         madx.options.info = True
#         madx.options.warn = True
#         madx.options.echo = True
                   
        madx.input('''
            select, flag=seqedit, class=collimator;
            select, flag=seqedit, class=hmonitor;
            select, flag=seqedit, class=vmonitor;
            select, flag=seqedit, class=kicker;
            select, flag=seqedit, class=tkicker;
            select, flag=seqedit, class=dipedge;
            select, flag=seqedit, class=elseparator;

            seqedit, sequence=SIS100RING;
                remove, element=selected;
                flatten;
            endedit;

            select, flag=seqedit, class=marker;
            seqedit, sequence=SIS100RING;
                remove, element=selected;
                install, element=SIS100RING$START, s=0;
                flatten;
            endedit;
        ''')
        
        madx.use(sequence='sis100ring')
        
        madx.input('''
            match, sequence=SIS100RING;
            global, sequence=SIS100RING, q1={qx}, q2={qy};
            vary, name=kqf, step=0.00001;
            vary, name=kqd, step=0.00001;
            lmdif, calls=500, tolerance=1.0e-10;
            endmatch;
        '''.format(qx=qx, qy=qy)
        )
        
        twiss = madx.twiss();
        
        # ready to add magnet errors
        
        madx.command.readtable(file=filename_error_table, table="errors")
        errors = madx.table.errors

        relevant_errs = [category for category in errors.keys() 
                         if any(errors[category]) and category != 'name'
                         and category[:1] == 'k' and category[-1:] == 'l']

        for el_idx, elem_name in enumerate(errors.name):
            el = madx.elements[elem_name]
            for category in relevant_errs:
                err_val = errors[category][el_idx]
                if not err_val:
                    continue

                if not 's' in category:
                    # normal knl component
                    order = int(category[1:-1])
                    try:
                        el.knl[order] += err_val
                    except IndexError:
                        el.knl[order] = err_val
                else:
                    # skew ksl component
                    order = int(category[1:-2])
                    try:
                        el.ksl[order] += err_val
                    except IndexError:
                        el.ksl[order] = err_val
        
        madx.use(sequence='SIS100RING')

        madx.command.save(sequence='sis100ring', file=filename_error_table + '_full-lattice.seq')

        madx.command.stop()

# PySTL: read in lattice with errors in full definition

In [7]:
nturns = 20000 #2**16

p0c = 238*0.2 * 1e9

npart = 1000

In [8]:
epsx_rms_fin = 35e-6 / 4 # geometrical emittances
epsy_rms_fin = 15e-6 / 4

epsx_gauss = epsx_rms_fin #* 1.778
epsy_gauss = epsy_rms_fin #* 1.82

sig_z = 58 / 4. # in m
sig_dp = 0.5e-3

In [9]:
pystl_device = "opencl:0.1"

In [ ]:
e_seed = 1

for qx in tune_range_qx:
    for qy in tune_range_qy:
        qqx, qqy = int(np.round((qx%1) * 100)), int(np.round((qy%1) * 100))
        
        filename_error_table = "./errors_{qqx}_{qqy}_{eseed:d}".format(
            qqx=qqx, qqy=qqy, eseed=e_seed)
        
        if os.path.exists(filename_error_table + '_done'):
            continue
        
        print ('\n\n\n=== Running at Qx = {:.2f} and Qy = {:.2f} ===\n\n\n'.format(qx, qy))
        
        # read in MAD-X sequence with errors
        
        madx = Madx()
        madx.options.echo = False
        madx.options.warn = False
        madx.options.info = False
        
        madx.call(file=filename_error_table + '_full-lattice.seq')
        
        madx.command.beam(particle='ion', mass=238*nmass, charge=28, energy=238*0.2+238*nmass)
        
        madx.use(sequence='SIS100RING')
        
        twiss = madx.twiss();
        
        sis100 = madx.sequence.sis100ring
        
        assert 2 == len(
            [f.name for f in map(lambda x: x.base_type, sis100.expanded_elements) 
             if 'drift' != f.name and 'multipole' != f.name and 'rfcavity' != f.name])
        
        # go to PySTL
        
        elements = pyst.Elements.from_mad(sis100)

        elements.BeamMonitor(num_stores=nturns);

        particles = pyst.Particles.from_ref(npart, p0c=p0c, q0=28, mass0=238*nmass*1e9)
        
        # matching

        np.random.seed(987654321)

        x = np.random.normal(
            loc=0, scale=np.sqrt(twiss['betx'][0] * epsx_gauss), size=npart)
        xp = np.random.normal(
             loc=0, scale=np.sqrt(epsx_gauss / twiss['betx'][0]), size=npart)
        xp += -twiss['alfx'][0] / twiss['betx'][0] * x

        y = np.random.normal(
            loc=0, scale=np.sqrt(twiss['bety'][0] * epsy_gauss), size=npart)
        yp = np.random.normal(
             loc=0, scale=np.sqrt(epsy_gauss / twiss['bety'][0]), size=npart)
        yp += -twiss['alfy'][0] / twiss['bety'][0] * y

        z = np.random.normal(loc=0, scale=sig_z, size=npart)
        dp = np.random.normal(loc=0, scale=sig_dp, size=npart)

        particles.x *= 0
        particles.px *= 0
        particles.y *= 0
        particles.py *= 0
        particles.zeta *= 0
        particles.delta *= 0

        particles.x += x
        particles.px += xp
        particles.y += y
        particles.py += yp
        particles.zeta += z
        particles.delta += dp
        
        # PySTL job
        
        job = pyst.TrackJob(elements, particles, device=pystl_device)

        job.track(nturns)

        job.collect()
        
        # coordinates
#         np.save(filename_error_table + '_x.npy', job.output.particles[0].x)
#         np.save(filename_error_table + '_y.npy', job.output.particles[0].y)
#         np.save(filename_error_table + '_px.npy', job.output.particles[0].px)
#         np.save(filename_error_table + '_py.npy', job.output.particles[0].py)
#         np.save(filename_error_table + '_zeta.npy', job.output.particles[0].zeta)
#         np.save(filename_error_table + '_delta.npy', job.output.particles[0].delta)

        store = {}

        # statistics
        x = job.output.particles[0].x.reshape((nturns, npart)).T
        store['std_x'] = np.mean(np.std(x, axis=0)[-50:])
        y = job.output.particles[0].y.reshape((nturns, npart)).T
        store['std_y'] = np.mean(np.std(y, axis=0)[-50:])
        
        # losses
        pbuffer = job.particles_buffer.get_object(0)
        np.save(filename_error_table + '_alive.npy', pbuffer.state)
        np.save(filename_error_table + '_lost_at_element.npy', 
                pbuffer.at_element[~pbuffer.state.astype(bool)])
        np.save(filename_error_table + '_lost_at_turn.npy',
                pbuffer.at_turn[~pbuffer.state.astype(bool)])
        
        store['losses'] = np.sum(pbuffer.state)
        
        # finish job
        pickle.dump(store, open(filename_error_table + '_summary.p', 'wb'))

        !touch "$filename_error_table"_done
        
        del job, elements, particles




=== Running at Qx = 18.68 and Qy = 18.64 ===




  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.04.02  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2018.10.03               +
  + Execution date: 2019.06.04 15:51:11      +
  ++++++++++++++++++++++++++++++++++++++++++++
enter Twiss module

++++++ table: summ

            length             orbit5               alfa            gammatr 
            1083.6                 -0     0.004119836673        15.57973263 

                q1                dq1            betxmax              dxmax 
       18.68014324       -36.07764495        20.13606132        3.253066381 

             dxrms             xcomax             xcorms                 q2 
       1.397493659                  0                  0         18.6399252 

               dq2            betymax              dymax              dyrms 
      -42.54909298        21.82823356      0.03780627061      0.01309646836 

      

enter Twiss module

++++++ table: summ

            length             orbit5               alfa            gammatr 
            1083.6                 -0     0.004120971507        15.57758731 

                q1                dq1            betxmax              dxmax 
       18.67967795       -36.18016248         20.1575529        3.254647771 

             dxrms             xcomax             xcorms                 q2 
       1.397905669                  0                  0        18.70040185 

               dq2            betymax              dymax              dyrms 
      -42.52902961        21.39240831      0.03875635163      0.01307919542 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            synch_3            synch_4            synch_5 
                 0                  0                  0                  0 

            nflips 
          